In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
import os
import torch
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from keras.optimizers import SGD, Adam

In [35]:
# Load the training data
train_data = pd.read_csv('../../2nd-Comp-Data/train.csv')
test_data = pd.read_csv('../../2nd-Comp-Data/test.csv')

used = []

# Extract features and target variable
X = train_data.drop('price_doc', axis=1)
y = train_data['price_doc']
X_test = test_data.drop(['row ID'], axis=1)

In [36]:
# X = pd.get_dummies(X)
# X_test = pd.get_dummies(X_test) 

In [37]:
# drop all columns in X_train with dtypes object
for col in X.columns:
    if X[col].dtype == 'object':
        X.drop(col, axis=1, inplace=True)

# drop all columns in X_test with dtypes object
for col in X_test.columns:
    if X_test[col].dtype == 'object':
        X_test.drop(col, axis=1, inplace=True)

used.append("Removed Object Dtypes")

In [38]:
X = X.astype('float32')
X_test = X_test.astype('float32')

In [39]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [40]:
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
used.append("StandardScaler")

<h1>With Keras<h1>

In [41]:
features_used = X_train.shape[1]

optimizerUsing = SGD(learning_rate=0.01, momentum=0.9)

# Build the neural network
os.environ["KERAS_BACKEND"] = "torch"
model = Sequential()
model.add(Dense(100, input_dim=X_train.shape[1], activation='relu'))
# model.add(Dropout(0.3))  # You can use either dropout or early stopping
model.add(Dense(70, activation='relu'))
# model.add(Dropout(0.3))
model.add(Dense(1, activation='linear'))  # Output layer with linear activation for regression

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=1800, validation_data=(X_val, y_val), callbacks=[early_stopping])

used.append("Keras With Torch Backend")
# used.append("1 Hidden Layer: Hidden Layer 1 with 140 neurons relu activation, Output linear")
used.append("2 Hidden Layers: Hidden Layer 1 with 100 neurons relu activation, Hidden Layer 2 with 70 neurons relu activation, Dropout 0.3, Output linear")
used.append("Optimizer: SGD(learning_rate=0.01, momentum=0.9)")
used.append("Loss Calculation: Mean Squared Error")
used.append("EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)")
# used.append("batch_size=1800, epochs=100, early_stopping, dropout=0.3 (used in hidden layers only)")
used.append("batch_size=1800, epochs=100, early_stopping")

# Make predictions
predictions = model.predict(X_test)

Epoch 1/100
71/71 [==============================] - 0s 4ms/step - loss: 694020141481984.0000 - val_loss: 694848533299200.0000
Epoch 2/100
71/71 [==============================] - 0s 3ms/step - loss: 693921155907584.0000 - val_loss: 694634456023040.0000
Epoch 3/100
71/71 [==============================] - 0s 3ms/step - loss: 693507631087616.0000 - val_loss: 693971152011264.0000
Epoch 4/100
71/71 [==============================] - 0s 3ms/step - loss: 692507977449472.0000 - val_loss: 692582938050560.0000
Epoch 5/100
71/71 [==============================] - 0s 3ms/step - loss: 690656913653760.0000 - val_loss: 690226074746880.0000
Epoch 6/100
71/71 [==============================] - 0s 4ms/step - loss: 687721370615808.0000 - val_loss: 686672593289216.0000
Epoch 7/100
71/71 [==============================] - 0s 3ms/step - loss: 683486801297408.0000 - val_loss: 681732978245632.0000
Epoch 8/100
71/71 [==============================] - 0s 3ms/step - loss: 677787312586752.0000 - val_loss: 67525

<h1>With Skilearn<h1>

In [42]:
# features_used = X_train.shape[1]

# # Build the neural network using MLPRegressor from scikit-learn
# model = MLPRegressor(hidden_layer_sizes=(140), activation='relu', solver='adam', max_iter=100,
#                      batch_size=1800, early_stopping=True, random_state=42)

# # Train the model
# model.fit(X_train, y_train)

# used.append("MLPRegressor")
# used.append("1 Hidden Layer: Hidden Layer 1 with 140 neurons relu activation, Output linear")
# # used.append("2 Hidden Layers: Hidden Layer 1 with 128 neurons relu activation, \nHidden Layer 2 with 64 neurons relu activation, Output linear")
# used.append("Adam Optimizer")
# # used.append("Loss Calculation: Mean Squared Error")
# used.append("EarlyStopping")
# used.append("batch_size=1800, epochs=100, early_stopping, dropout=0.3 (used in hidden layers only)")

# # Make predictions
# predictions = model.predict(X_test)

In [43]:
# Create a DataFrame with the results
submission_df = pd.DataFrame({'row ID': test_data['row ID'], 'price_doc': predictions.flatten()})

# Save the results to a CSV file
submission_df.to_csv('Day7.3.csv', index=False)

In [44]:
print("Features Used = " + str(features_used) + "\n")
for i in used:
    print(i)

Features Used = 256

Removed Object Dtypes
StandardScaler
Keras With Torch Backend
2 Hidden Layers: Hidden Layer 1 with 100 neurons relu activation, 
Hidden Layer 2 with 70 neurons relu activation, Dropout 0.3, Output linear
Optimizer: <keras.src.optimizers.sgd.SGD object at 0x297551690>
Loss Calculation: Mean Squared Error
EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
batch_size=1800, epochs=100, early_stopping
